# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [52]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
import nltk
nltk.download('rslp')

[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\rafae\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [53]:
stemmer = nltk.stem.RSLPStemmer()

In [68]:
def limpa_tweet(tweet):
    tweet = tweet.replace(" por ", " ")
    tweet = tweet.replace(" do ", " ")
    tweet = tweet.replace(" da ", " ")
    tweet = tweet.replace(" de ", " ")
    tweet = tweet.replace(" '", "")
    tweet = tweet.replace("' ", "")
    tweet = tweet.replace("#", "")
    tweet = tweet.replace(" à ", " ")
    tweet = tweet.replace(" aos ", " ")
    tweet = tweet.replace(" o ", " ")
    tweet = tweet.replace(" a ", " ")
    tweet = tweet.replace(" e ", " ")
    tweet = tweet.replace("!", " ")
    tweet = tweet.replace("?", " ")
    tweet = tweet.replace("(", " ")
    tweet = tweet.replace(")", " ")
    tweet = tweet.replace(" rt ", " ")
    tweet = tweet.replace("rt ", " ")

    tweet = tweet.lower()
    tweet = tweet.split()
    
    for palavra in tweet:
        if '@' in palavra or 'https' in palavra:
            tweet.remove(palavra)

                          
    tweets_stemizados = []
    for palavra in tweet:
        tweets_stemizados.append(stemmer.stem(palavra))

    return tweets_stemizados

In [95]:
dados = pd.read_excel('tweets_certos_3.0.xlsx')

In [96]:
total_0 = dados['Treinamento'][dados['rel/irrel'] == 0].count()
total_1 = dados['Treinamento'][dados['rel/irrel'] == 1].count()

total = total_0 + total_1

In [97]:
p_0 = total_0 / total
p_1 = total_1 / total

print("Probabilidade de Irrelevante: {:.2f}%".format(p_1*100))
print("Probabilidade de Relevante: {:.2f}%".format(p_0*100))

Probabilidade de Irrelevante: 31.79%
Probabilidade de Relevante: 68.21%


In [122]:
palavras_relevantes = []
palavras_irrelevantes = []

frequencia_irrelevantes = {}
frequencia_relevantes = {}

n_relevante = 0
n_irrelevante = 0

i = 0
for avaliacao in dados['rel/irrel']:
    tweet_limpo = limpa_tweet(dados['Treinamento'][i])
    for palavra in tweet_limpo:
        if avaliacao == 1:
            if palavra not in palavras_irrelevantes:
                palavras_irrelevantes.append(palavra)
                frequencia_irrelevantes[palavra] = 1
            else:
                frequencia_irrelevantes[palavra] += 1
            n_irrelevante += 1
            
        else:
            if palavra not in palavras_relevantes:
                palavras_relevantes.append(palavra)
                frequencia_relevantes[palavra] = 1
            else:
                frequencia_relevantes[palavra] += 1
            n_relevante += 1
            
    i += 1
    
n_total = n_relevante + n_irrelevante


In [99]:
def calc_prob(tweet, dic_palavras, p, n_palavras):
    prob = p
    total = n_palavras
    tweet_limpo = limpa_tweet(tweet)
    for palavra in tweet_limpo:
        ocorrencia = 1
        if palavra in dic_palavras:
            ocorrencia += dic_palavras[palavra]
        prob *= (ocorrencia/total)
    return prob


In [100]:
def comparador_prob(prob_0, prob_1):
    if prob_0 > prob_1:
        return 0
    return 1


In [101]:
#================================================Teste de funcionamento=========================================================
tweet = input('Digite seu tweet de teste: ')
prob_1 = calc_prob(tweet, frequencia_irrelevantes, p_1, n_irrelevante)
prob_2 = calc_prob(tweet, frequencia_relevantes, p_0, n_relevante)
resp = comparador_prob(prob_0, prob_1)
print(limpa_tweet(tweet))
if resp == 1:
    print('relevante')
if resp == 0:
    print('irrelevante')

Digite seu tweet de teste: d
['d']
relevante


In [118]:
dados_teste = pd.read_excel("tweets_certos_3.0.xlsx", sheet_name=1)

In [119]:
coluna_previsao = []
for tweet in dados_teste['Teste']:
    prob_1 = calc_prob(tweet, frequencia_irrelevantes, p_1, n_irrelevante)
    prob_0 = calc_prob(tweet, frequencia_relevantes, p_0, n_relevante)
    coluna_previsao.append(comparador_prob(prob_0, prob_1))
    
dados_teste['Previsão'] = coluna_previsao

In [120]:
acertos = 0
i = 0
for avaliacao in dados_teste['rel/irrel']:
    if avaliacao == dados_teste['Previsão'][i]:
        acertos += 1
    i += 1
    
print("Porcentagem de Acerto: {:.2f}%".format(100*acertos/len(dados_teste)))

Porcentagem de Acerto: 78.00%


In [105]:
dados_teste

,Teste,rel/irrel,Previsão
0,e ainda há quem ache certo o @twitter copiar i...,0,1
1,@ivoneuman @twitter pior é ver tuíte patrocina...,0,0
2,"rt @cesarfilho: na sua opinião, quem vence as ...",1,1
3,@twitter @twitterlatam @twitterbrasil \n#otwit...,0,0
4,rt @okikoloko: @canaltech sério!!! tem alguém ...,0,0
5,rt @srtalawyerbr17: assistimos a censura e a p...,1,1
6,rt @andremcapanema: @luscas alguém avisa pra e...,0,0
7,@3dollarsminsuga @twitter eu n te xinguei kffj...,0,1
8,@jimi_ssi @twitter hahahahahaha eu também entr...,0,1
9,@odiodobem2 o mais legal é q o @facebook @twit...,0,1


___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).
